In [ ]:
import h5py, sys, os, time
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm

In [ ]:
import spimage

In [ ]:
import sys; sys.path.append("../../offline/")
import sparse, geom, utils

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

In [ ]:
emc_folder   = path_to_recons + "sucrose_0000/"
emc_output   = emc_folder + "data/r2/output_001_%03d.h5"
emc_photons  = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_2145_lowq5.h5"

In [ ]:
photons_list = emc_folder + str(np.loadtxt(emc_photons, dtype=str))

## Load initial model

In [ ]:
with h5py.File(emc_folder + 'data/r2/init.h5', 'r') as f:
    dsamples = f['dsamples'][:]
    intensity = f['intensity'][...]
    pixelsize = f['pixelsize'][...]
    wavelength = f['wavelength'][...]
    distance = f['distance'][...]
    material = str(f['material'][...])

In [ ]:
rescale = np.empty(5, dtype='float')
rescale[0] = 7.897398e-02
rescale[1] = 7.897398e-01
rescale[2] = 7.897398e+00
rescale[3] = 7.897398e+01
rescale[4] = 7.897398e+02

In [ ]:
intensity = np.empty(5, dtype='float')
intensity[0] = 1e8
intensity[1] = 1e7
intensity[2] = 1e6
intensity[3] = 1e5
intensity[4] = 1e4

## Load EMC reconstruction

In [ ]:
n = 1

In [ ]:
nr_rot = 180
nr_static = 0
with h5py.File(emc_output %n,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]
    scale = f['scale'][:]
    mutual_info = f['mutual_info'][:]
nr_modes = occupancies.shape[1] - nr_static
modes = orientations % (nr_modes)
rotind = orientations // (nr_modes)
modes[rotind >= nr_rot] = orientations[rotind >= nr_rot] - nr_modes * (nr_rot - 1)
blacklisted = (occupancies == 0).all(axis=1)
modes[blacklisted] = -1
ndata = len(modes) - blacklisted.sum() 

In [ ]:
N = occupancies.shape[1]
ncols = 20
nrows = N//ncols
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4,nrows*4), dpi=100)
for i in range(N):
    c,r = i//ncols, i%ncols
    axes[c,r].axis('off')
    nclass = ((modes==i) & (~blacklisted)).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
    axes[c,r].text(0,0, "%d" %i, color='r')
plt.show()

In [ ]:
det = detector.Detector(emc_detector, mask_flag=True)
emc = reademc.EMCReader(photons_list, det)

In [ ]:
diameters = np.zeros(occupancies.shape[0])
for i in range(len(dsamples)):
    diameters[modes==i] = dsamples[i]*1e9
diameters[modes==-1] = np.nan

In [ ]:
cut = int(ndata*0.90)
threshold = np.sort(likelihood[~blacklisted])[::-1][cut]
print(threshold)
select = ((likelihood>threshold) & (likelihood!=0))

## Create static averages

In [ ]:
boxsize = (181,181)
fr = emc.get_frame(0)
ymin = np.where(fr.mask==0)[0].min() - 3
xmin = np.where(fr.mask==0)[1].min() - 3
mask = (fr.mask[ymin:ymin+boxsize[0],xmin:xmin+boxsize[1]] == 0)[::-1,::-1]
crop = lambda img: img[ymin:ymin+boxsize[0],xmin:xmin+boxsize[1]][::-1,::-1]

In [ ]:
averages = np.empty((nr_modes, boxsize[0], boxsize[1]))
for m in range(nr_modes):
    if (modes==m).sum():
        averages[m] = crop(np.mean([emc.get_frame(i) for i in np.where((modes==m) & select)[0]], axis=0))

In [ ]:
# customized colors
cmap = cm.get_cmap('cubehelix')
cmap.set_bad('0.75')
cmap.set_under('black')

In [ ]:
fig, axes = plt.subplots(ncols=20, nrows=5, figsize=(20,5), dpi=200)
for i in range(nr_modes):
    c,r = i//20, i%20
    axes[c,r].axis('off')
    axes[c,r].set_title("%d: %d nm" %(i,dsamples[i]*1e9), fontsize=5, pad=0.1)
    axes[c,r].imshow(averages[i], norm=colors.LogNorm(), cmap=cmap)
plt.show()

## Radial averages

In [ ]:
Raverages = []
for i in range(nr_modes):
    center, radial = spimage.radialMeanImage(averages[i], msk=mask, 
                                             cx=boxsize[0]//2, cy=boxsize[1]//2, output_r=True)
    Raverages.append(radial)
Raverages = np.array(Raverages)

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
plt.title("Radial averages of static 2D averages")
plt.imshow(Raverages, aspect='auto', norm=colors.LogNorm(vmin=1e-5, vmax=10), 
           cmap='viridis', interpolation='nearest',
           extent=[center[0],center[-1],dsamples[-1]*1e9,dsamples[0]*1e9])
plt.xlabel("Spatial frequency [px]")
plt.ylabel("Particle size [nm]")
plt.show()

## Fitting the average intensity

In [ ]:
import scipy.optimize

In [ ]:
qr = spimage.x_to_qx(center, pixelsize, distance)

In [ ]:
def sphere_model(I,D):
    A = spimage.sphere_model_convert_intensity_to_scaling(I, D, wavelength, pixelsize, 
                                                          distance, material=material)
    s = spimage.sphere_model_convert_diameter_to_size(D, wavelength, pixelsize, distance)
    return spimage.I_sphere_diffraction(A,qr,s)

In [ ]:
fluence = np.empty(nr_modes)
Faverages = []
for m in range(nr_modes):
    fitfunc  = lambda i: sphere_model(intensity[n]*i,dsamples[m])
    costfunc = lambda i: np.sum((fitfunc(i)[5:] - Raverages[m][5:])**2)
    res = scipy.optimize.minimize(costfunc, 1.)
    fluence[m] = np.abs(res['x'][0])
    Faverages.append(fitfunc(res['x']))
Faverages = np.array(Faverages)

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
plt.title("Radial fits of static 2D averages")
plt.imshow(Faverages, aspect='auto', norm=colors.LogNorm(vmin=1e-5, vmax=10), 
           cmap='viridis', interpolation='nearest',
           extent=[center[0],center[-1],dsamples[-1]*1e9,dsamples[0]*1e9])
plt.xlabel("Spatial frequency [px]")
plt.ylabel("Particle size [nm]")
plt.show()

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
plt.title("Quadratic distance between radial fit and average")
plt.imshow((Faverages-Raverages)**2, aspect='auto', cmap='hot', 
           interpolation='nearest', norm=colors.LogNorm(vmin=1e-6, vmax=1),
           extent=[center[0],center[-1],dsamples[-1]*1e9,dsamples[0]*1e9])
plt.xlabel("Spatial frequency [px]")
plt.ylabel("Particle size [nm]")
plt.colorbar()
plt.show()

In [ ]:
plt.plot(dsamples*1e9, np.sum((Faverages-Raverages)**2, axis=1))

In [ ]:
plt.plot(dsamples*1e9, fluence)

In [ ]:
m = 75
plt.figure(figsize=(6,3), dpi=100)
plt.title("m = %d | size: %d nm" %(m,dsamples[m]*1e9))
plt.plot(center, Raverages[m], 'bo:', ms=2)
plt.plot(qr, Faverages[m], 'r-')
#plt.semilogy()
plt.ylim(0,2)
plt.xlim(0,100)
plt.show()

## Size/intensity distributions

In [ ]:
meanscale = np.array([scale[modes==m][select[modes==m]].mean() for m in range(nr_modes)])

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(diameters[~select], scale[~select], c='r', s=10, alpha=1)
plt.scatter(diameters[select], scale[select], c=likelihood[select], s=10, alpha=1, vmin=-2000,vmax=0)
plt.plot(dsamples*1e9, meanscale, color='k', lw=2)
plt.colorbar(label='Likelihood')
plt.xlim(50,250)
plt.ylim(0,5)
plt.ylabel("EMC fluence scale [arb. unit]")
plt.xlabel("Diameter [nm]")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist2d(diameters[select], scale[select], range=((dsamples[0]*1e9, dsamples[-1]*1e9),(0,5)), bins=len(dsamples), 
           norm=colors.LogNorm(vmin=1,vmax=200), cmap='inferno')
plt.plot(dsamples*1e9, meanscale, color='lightblue', lw=2)
plt.colorbar()
plt.ylabel("EMC fluence scale [arb. unit]")
plt.xlabel("Diameter [nm]")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(dsamples*1e9, meanscale, label="Mean EMC fluence")
plt.plot(dsamples*1e9, meanscale*0.60, label="Mean EMC fluence x 0.60")
plt.plot(dsamples*1e9, fluence, label="Fitted fluence")
plt.ylabel("Fluence scales")
plt.xlabel("Diameter [nm]")
plt.legend(frameon=0)
plt.show()

In [ ]:
intensities = scale*0.60*intensity[n]*1e-6

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(15,5), dpi=200)
axes[0].set_title("Size vs. Fluence")
axes[0].hist2d(diameters[select], intensities[select], range=((dsamples[0]*1e9, dsamples[-1]*1e9),(0,20)), bins=len(dsamples), 
           norm=colors.LogNorm(vmin=1,vmax=100), cmap='inferno')
axes[0].set_xlim(50,250)
axes[0].set_ylim(0,20)
axes[0].set_ylabel(r"Estimated fluence [$\mathrm{\mu J}/\mathrm{\mu} m^2$]")
axes[0].set_xlabel("Diameter [nm]")
#axes[0].axhline(6.5, color='b')
axes[1].set_title("Size histogram")
axes[1].hist(diameters[select], range=(dsamples[0]*1e9, dsamples[-1]*1e9), bins=len(dsamples), color='0.75')
#axes[1].hist(diameters[select & (intensities>6.5)], range=(dsamples[0]*1e9, dsamples[-1]*1e9), bins=len(dsamples)//1, color='b')
axes[1].set_xlabel("Diameter [nm]")
axes[2].set_title("Fluence histogram")
axes[2].hist(scale[select]*0.65*intensity[n]*1e-6, range=(0,20), bins=len(dsamples), color='r')
axes[2].set_xlabel(r"Estimated fluence [$\mathrm{\mu J}/\mathrm{\mu} m^2$]")
plt.show()

## Fit to individial patterns

In [ ]:
isort = np.sort(intensities[select])[::-1]

In [ ]:
k = -3
j = np.where(intensities==isort[k])[0][0]
center, data = spimage.radialMeanImage(crop(emc.get_frame(j)), msk=mask, 
                                             cx=boxsize[0]//2, cy=boxsize[1]//2, output_r=True)

In [ ]:
plt.figure()
plt.title(r"size: %d nm | fluence: %.2f $\mathrm{\mu J}/\mathrm{\mu m}^2$" %(diameters[j], intensities[j]))
plt.plot(center, sphere_model(intensities[j]*1e6, diameters[j]*1e-9),'r-')
plt.plot(center,data, 'bo:', ms=2)
plt.xlabel("Spatial frequency [px]")
plt.ylabel("Nr. of photons / px")
plt.show()